<a href="https://colab.research.google.com/github/HumaHamid92/humahh/blob/main/Time_series_classification_Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Model, load_model, save_model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM,  Bidirectional, Activation
from keras.layers.merge import concatenate
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/subject_1.csv')

In [ ]:
df_d = df.to_numpy()
df.head()

In [ ]:
def create_dataset(df_d, look_back=1):
	dataX, dataY = [], []
	for i in range(len(df_d)-look_back):
		a = df_d[i:(i+look_back), 0:12]
		dataX.append(a)
		dataY.append(df_d[i + look_back, 12])
	return np.array(dataX), np.array(dataY)

In [ ]:
# split into train and test sets
train_size = int(len(df_d) * 0.8)
test_size = len(df_d) - train_size
train, test = df_d[0:train_size,:], df_d[train_size:len(df_d),:]

In [ ]:
trainX.shape, trainy.shape, testX.shape, testy.shape

In [ ]:
num_classes = len(np.unique(trainy))
trainy = tf.keras.utils.to_categorical(trainy)
testy = tf.keras.utils.to_categorical(testy)

In [ ]:

model = Sequential()
model.add(Bidirectional(LSTM(64,  input_shape=(trainX.shape[1], trainX.shape[2]), activation='relu', return_sequences=True)))
model.add(Bidirectional(LSTM(64, activation='relu', return_sequences=False)))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
print(model.summary())

In [ ]:
epochs = 500 
batch_size = 150

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, verbose=1, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=50, min_lr=0.001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=80, verbose=1, baseline=0.4),
]
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy", tf.keras.metrics.Precision(name = 'precision'), tf.keras.metrics.Recall(name = 'recall')
    ],
)
history = model.fit(
    trainX,
    trainy,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2, 
    verbose=1,
)

In [ ]:
rounded_labels=np.argmax(testy, axis=1)
rounded_labels[1]

In [ ]:
y_pred = np.argmax(model.predict(testX),axis=1)
accuracy = accuracy_score(rounded_labels, y_pred)
precision = precision_score(rounded_labels, y_pred, zero_division=1)
recall = recall_score(rounded_labels, y_pred, zero_division=1)
print('Accuracy: ', accuracy*100)
print('Precision: ', precision*100)
print('Recall: ', recall*100)

In [ ]:
metric = "accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()
print("Train accuracy", history.history[metric][9])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()